In [1]:
import numpy as np
from scipy.interpolate import interp2d, SmoothBivariateSpline, RectBivariateSpline
import matplotlib.pyplot as plt
from matplotlib import cm
import plotly.graph_objects as go

t_points = np.load("./data.npy")
peaks = np.load("./peaks.npy")

In [2]:
# some analysis of the data

idx = 160386 # a position without any pressure
xs, ys, zs = t_points[idx, :, :, 0], t_points[idx, :, :, 1], t_points[idx, :, :, 2]

# compute the delta between markers in x and y (median/mean)
dx = np.median(np.diff(xs,axis=0))
dy = np.median(np.diff(ys,axis=1))

w, h = 170, 170 # widht and height of the sensor (in mm) -> Note, however, the sensor is not square 
# we assume it is square for now!
cx, cy = w/2, h/2 # center of the sensor

cw, ch = dx*(xs.shape[0]-1), dy*(ys.shape[1]-1) # width and height of the center area of the sensor (where markers are)

# side offset in x and y
ox, oy = (w-cw)/2, (h-ch)/2

In [3]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure(data=
    [go.Scatter3d(x=xs.flatten()+ox, y=ys.flatten()+oy, z=zs.flatten(), mode="markers", marker=dict(color="red", size=4))]
)

fig.update_layout(
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="z",
        xaxis=dict(range=[0, w]),
        yaxis=dict(range=[0, h]),
        zaxis=dict(range=[-1, 20]),
    )
)

fig.show()

In [4]:
from scipy.optimize import minimize

# defining the minimization/interpolation function
def min_fun(p, xs, ys, zs, w,h) -> float:
    z = np.zeros((3,3))
    z[1,1] = p[2]
    f = interp2d([0, p[0], w], [0, p[1], h], z, kind='linear')
    zs_ = np.zeros_like(zs.flatten())
    for i in range(len(zs_)):
        zs_[i] = f(xs.flatten()[i], ys.flatten()[i])

    return np.sum(((zs.flatten() - zs_)**2))

# Reconstruct (with all) the data from the measurements

In [5]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

selected_peaks = [100, 300, 345, 346]

fig = make_subplots(rows=1, cols=len(selected_peaks), 
    specs=[[{"type":"scene"}]*len(selected_peaks)], subplot_titles=[f"Peak {i}" for i in selected_peaks],
    )


for i, peak_id in enumerate(selected_peaks):

    idx = peaks[peak_id] # a position with pressure
    xs, ys, zs = t_points[idx, :, :, 0]+ox, t_points[idx, :, :, 1]+oy, t_points[idx, :, :, 2]

    # the starting point (x0) must be within the range of the sensors! Otherwise the minimization will fail.
    res = minimize(min_fun, x0=[cx,cy,zs[1,1]], args=(xs, ys, zs, w,h), bounds=[(ox,w-ox),(oy,h-oy),(0,20)])
    print(f"optimization at {res.x} with an error of {res.fun}.")


    # Create figure
    fig.add_trace(
        go.Scatter3d(x=xs.flatten(), y=ys.flatten(), z=zs.flatten(), mode="markers", marker=dict(color="red", size=4), name='measurements'),
        row=1, col=i+1,
        
    )

    fig.add_trace(
        go.Scatter3d(x=[res.x[0]], y=[res.x[1]], z=[res.x[2]], mode="markers", marker=dict(color="blue", size=4), name='optimization'),        
        row=1, col=i+1,
    )

    fig.update_layout(
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="z",
            xaxis=dict(range=[0, w]),
            yaxis=dict(range=[0, h]),
            zaxis=dict(range=[-1, 20]),
        )
    )

fig.show()

optimization at [140.87376579  61.97710721  11.99755161] with an error of 102.70447068992587.
optimization at [109.5043799   75.60460061  13.82531326] with an error of 66.69324186643028.
optimization at [61.46670304 84.74603952 13.53100556] with an error of 70.4994802339582.
optimization at [86.02590177 85.07748133 13.84306127] with an error of 56.968457410392745.


# Reconstruct (a selection/masked) the data from the measurements

In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

selected_peaks = [0, 300, 345, 346]

fig = make_subplots(rows=1, cols=len(selected_peaks), 
    specs=[[{"type":"scene"}]*len(selected_peaks)], subplot_titles=[f"Peak {i}" for i in selected_peaks],
    )

mask = np.zeros_like(zs).astype(bool)
mask[::2,0] = True
mask[::2,-1] = True
mask[0,::2] = True
mask[-1,::2] = True

for i, peak_id in enumerate(selected_peaks):

    idx = peaks[peak_id] # a position with pressure
    xs, ys, zs = t_points[idx, :, :, 0]+ox, t_points[idx, :, :, 1]+oy, t_points[idx, :, :, 2]

    xs_ = xs[mask]
    ys_ = ys[mask]
    zs_ = zs[mask]

    # the starting point (x0) must be within the range of the sensors! Otherwise the minimization will fail.
    res = minimize(min_fun, x0=[cx,cy,zs[1,1]], args=(xs_, ys_, zs_, w,h), bounds=[(ox,w-ox),(oy,h-oy),(0,20)])
    print(f"optimization at {res.x} with an error of {res.fun}.")


    # Create figure
    fig.add_trace(
        go.Scatter3d(x=xs.flatten(), y=ys.flatten(), z=zs.flatten(), mode="markers", marker=dict(color="black", size=1), name='measurements'),
        row=1, col=i+1 )

        
    fig.add_trace(
        go.Scatter3d(x=xs_.flatten(), y=ys_.flatten(), z=zs_.flatten(), mode="markers", marker=dict(color="red", size=4), name='measurements'),
        row=1, col=i+1 )

    fig.add_trace(
        go.Scatter3d(x=[res.x[0]], y=[res.x[1]], z=[res.x[2]], mode="markers", marker=dict(color="blue", size=4), name='optimization'),        
        row=1, col=i+1 )

    fig.update_layout(
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="z",
            xaxis=dict(range=[0, w]),
            yaxis=dict(range=[0, h]),
            zaxis=dict(range=[-1, 20]),
        )
    )

fig.show()

optimization at [41.55762198 52.36863163  9.52315311] with an error of 5.8069358726558855.
optimization at [113.87689426  77.37557507  10.41801015] with an error of 2.933278370749415.
optimization at [54.84412677 85.70148203 10.26720217] with an error of 2.2876876006892704.
optimization at [81.11076047 84.41841947 10.33684928] with an error of 1.892280758369943.
